In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')

In [4]:
md = pd.read_csv("train500.csv")
print(md.shape)
md.head()

(499, 12)


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [25]:
groups = md[md['impressions'].notnull()]

In [26]:
city = groups['city']
hotels = groups['impressions']

In [27]:
groups[['city','impressions','reference','action_type']]

,city,impressions,reference,action_type
13,"Sydney, Australia",3400638|1253714|3367857|5100540|1088584|666916...,109038,clickout item
15,"Sydney, Australia",55109|129343|54824|2297972|109014|1257342|1031...,1257342,clickout item
115,"Krakow, Poland",2795374|5582964|1088390|2781070|1258068|127196...,2795374,clickout item
121,"London, United Kingdom",12693|46363|81657|18448|47687|152913|18417|927...,1032816,clickout item
122,"London, United Kingdom",12693|46363|81657|18448|47687|152913|18417|927...,1032816,clickout item
176,"Bakersfield, USA",1306936|56482|2842358|6881276|65685|63259|6539...,65685,clickout item
177,"Bakersfield, USA",1306936|56482|2842358|6881276|65685|63259|6539...,1320460,clickout item
180,"Legian, Indonesia",1258184|3866722|8929970|2315702|116619|1511641...,3143258,clickout item
181,"Mazamitla, Mexico",2349076|2552514|7159866|3898458|1846017|725025...,2552514,clickout item
184,"Alicante, Spain",6721|6724|40109|147227|80983|6719|40718|119405...,110591,clickout item


In [46]:
popHotel = md.groupby('reference').count()
popHotel.head()

,user_id,session_id,timestamp,step,action_type,platform,city,device,current_filters,impressions,prices
reference,,,,,,,,,,,
10091602,6,6,6,6,6,6,6,6,0,3,3
1032816,4,4,4,4,4,4,4,4,0,2,2
104499,3,3,3,3,3,3,3,3,0,2,2
1045290,37,37,37,37,37,37,37,37,0,0,0
109038,8,8,8,8,8,8,8,8,0,1,1


In [61]:
popHotel.loc[popHotel.index.str.isnumeric()==True]

,user_id,session_id,timestamp,step,action_type,platform,city,device,current_filters,impressions,prices
reference,,,,,,,,,,,
10091602,6,6,6,6,6,6,6,6,0,3,3
1032816,4,4,4,4,4,4,4,4,0,2,2
104499,3,3,3,3,3,3,3,3,0,2,2
1045290,37,37,37,37,37,37,37,37,0,0,0
109038,8,8,8,8,8,8,8,8,0,1,1
1091180,24,24,24,24,24,24,24,24,2,2,2
110591,1,1,1,1,1,1,1,1,0,1,1
1257342,1,1,1,1,1,1,1,1,0,1,1
1258184,1,1,1,1,1,1,1,1,0,0,0


In [78]:
cityHotels = groups[['city','impressions']]

In [79]:
def make_encoding_label_dict(col_unique):
    encoded_dict = {}
    for idx, unique in enumerate(col_unique):
        encoded_dict[unique] = idx + 1

    return encoded_dict


# get new encoding result
def get_newcode(key, label_dict):
    com_len = len(label_dict)
    if key == np.nan:
        return
    else:
        if key in label_dict:
            return label_dict[key]
        else:
            return com_len + 1

In [80]:
encoded_city_dict = make_encoding_label_dict(cityHotels.city.value_counts().index.tolist())

In [81]:
encoded_city_dict

{'Seoul, South Korea': 1,
 'Las Vegas, USA': 2,
 'Veracruz Llave, Mexico': 3,
 'Yekaterinburg, Russia': 4,
 'Sydney, Australia': 5,
 'London, United Kingdom': 6,
 'São Paulo, Brazil': 7,
 'Bolzano, Italy': 8,
 'Bakersfield, USA': 9,
 'Panama City, Panama': 10,
 'Krakow, Poland': 11,
 'Parnaíba, Brazil': 12,
 'Solihull, United Kingdom': 13,
 'Curitiba, Brazil': 14,
 'Amsterdam, Netherlands': 15,
 'Bengaluru, India': 16,
 'Mazamitla, Mexico': 17,
 'Legian, Indonesia': 18,
 'Alicante, Spain': 19,
 'St Georgen im Attergau, Austria': 20}

In [82]:
cityHotels['city'] = cityHotels['city'].apply(lambda x: get_newcode(x, encoded_city_dict))

In [83]:
cityHotels

,city,impressions
13,5,3400638|1253714|3367857|5100540|1088584|666916...
15,5,55109|129343|54824|2297972|109014|1257342|1031...
115,11,2795374|5582964|1088390|2781070|1258068|127196...
121,6,12693|46363|81657|18448|47687|152913|18417|927...
122,6,12693|46363|81657|18448|47687|152913|18417|927...
176,9,1306936|56482|2842358|6881276|65685|63259|6539...
177,9,1306936|56482|2842358|6881276|65685|63259|6539...
180,18,1258184|3866722|8929970|2315702|116619|1511641...
181,17,2349076|2552514|7159866|3898458|1846017|725025...
184,19,6721|6724|40109|147227|80983|6719|40718|119405...


In [90]:
#pd.Series.str.cat(
c1 = cityHotels.groupby('city')
c1.head()

,city,impressions
13,5,3400638|1253714|3367857|5100540|1088584|666916...
15,5,55109|129343|54824|2297972|109014|1257342|1031...
115,11,2795374|5582964|1088390|2781070|1258068|127196...
121,6,12693|46363|81657|18448|47687|152913|18417|927...
122,6,12693|46363|81657|18448|47687|152913|18417|927...
176,9,1306936|56482|2842358|6881276|65685|63259|6539...
177,9,1306936|56482|2842358|6881276|65685|63259|6539...
180,18,1258184|3866722|8929970|2315702|116619|1511641...
181,17,2349076|2552514|7159866|3898458|1846017|725025...
184,19,6721|6724|40109|147227|80983|6719|40718|119405...


In [113]:
c2 = cityHotels.groupby('city')['impressions'].apply(lambda x: ('|'+x).sum())
c2

city
1     |2802232|2733571|5477718|155374|155465|3549258...
2     |8561|8564|8568|8586|8588|102080|8613|55677|57...
3     |43007|42365|42884|349051|489031|488961|102843...
4     |1045290|3135752|1372961|1369024|3384308|13710...
5     |3400638|1253714|3367857|5100540|1088584|66691...
6     |12693|46363|81657|18448|47687|152913|18417|92...
7     |104499|104710|104724|164677|960133|5816566|87...
8     |22953|22951|22947|22949|8774118|45527|82900|2...
9     |1306936|56482|2842358|6881276|65685|63259|653...
10    |2603858|1765201|1844107|99445|99438|2219728|9...
11    |2795374|5582964|1088390|2781070|1258068|12719...
12                     |2035675|4095738|4933410|9167996
13    |47767|959737|3191226|1362528|1950827|1504481|...
14    |9172186|6816668|4497388|1827505|2066390|27291...
15    |921237|147262|42194|9051714|42109|42125|35832...
16    |9164598|1400686|2216204|2817576|4775794|56842...
17    |2349076|2552514|7159866|3898458|1846017|72502...
18    |1258184|3866722|8929970|2315702|1166

In [114]:
hotels = np.asarray(c2.values)

In [115]:
set(hotels[0].split('|'))

{'',
 '10091602',
 '1176048',
 '1394008',
 '155374',
 '155465',
 '155989',
 '156621',
 '1627467',
 '1824477',
 '1891895',
 '2267026',
 '2555968',
 '2727688',
 '2733571',
 '2736998',
 '2738638',
 '2802232',
 '2855981',
 '2867638',
 '3126684',
 '3136052',
 '3145303',
 '3171165',
 '3181886',
 '3367204',
 '3499270',
 '3501450',
 '3501452',
 '3505382',
 '3549258',
 '363046',
 '363061',
 '377811',
 '3825178',
 '385916',
 '3899082',
 '390361',
 '3954788',
 '4454362',
 '4638538',
 '475101',
 '475131',
 '475206',
 '475256',
 '4773608',
 '4849224',
 '4871108',
 '5109850',
 '5197174',
 '5477718',
 '5645106',
 '5677278',
 '5702366',
 '6508748',
 '6523164',
 '6625240',
 '6800422',
 '6833196',
 '6893402',
 '7016734',
 '797516',
 '8143802',
 '8403088',
 '844336',
 '8444298',
 '9033886',
 '9386776',
 '9776792'}